# Edge Transit Log Analysis

Fab별 반송 효율 분석을 위한 노트북

## 분석 가능 항목
- Throughput (시간당 처리량)
- Transit Time (Edge 통과 시간)
- Vehicle 활용률
- Edge별 혼잡도
- Fab간 비교 분석
- 여러 실험 결과 비교

## 1. Setup & Load Data

In [ ]:
import struct
import polars as pl
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from collections import defaultdict
from pathlib import Path

print('Setup complete!')
print(f'Polars version: {pl.__version__}')

In [ ]:
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# 로그 파일 경로 설정
LOG_DIR = Path('/content/drive/MyDrive/vps_logs')  # 필요시 수정
print(f'Log directory: {LOG_DIR}')

# 파일 목록 확인
if LOG_DIR.exists():
    log_files = list(LOG_DIR.glob('*.bin'))
    print(f'Found {len(log_files)} log files:')
    for f in log_files:
        size_kb = f.stat().st_size / 1024
        print(f'  - {f.name} ({size_kb:.1f} KB)')
else:
    print(f'Directory not found: {LOG_DIR}')
    print('Create the directory and upload your .bin files')

In [ ]:
# Binary Format
RECORD_SIZE = 28
RECORD_FORMAT = '<I BB H I I I f B 3x'  # little-endian

EDGE_TYPES = {
    0: 'LINEAR',
    1: 'CURVE_90',
    2: 'CURVE_180',
    3: 'CURVE_CSC',
    4: 'S_CURVE',
    5: 'LEFT_CURVE',
    6: 'RIGHT_CURVE',
}

# Edge Type별 기대 속도 (m/s) - 직선이 더 빠름
EXPECTED_SPEEDS = {
    'LINEAR': 3.0,      # 직선: 3.0 m/s
    'CURVE_90': 2.2,    # 곡선: 더 느림
    'CURVE_180': 2.0,
    'CURVE_CSC': 2.2,
    'S_CURVE': 2.3,
    'LEFT_CURVE': 2.2,
    'RIGHT_CURVE': 2.2,
}

def format_edge_id(edge_id):
    """Edge ID를 E0001 형식으로 포맷팅"""
    return f"E{edge_id+1:04d}"

def parse_log_file(filepath):
    """바이너리 로그 파일을 Polars DataFrame으로 변환"""
    records = []
    
    with open(filepath, 'rb') as f:
        data = f.read()
    
    for i in range(0, len(data), RECORD_SIZE):
        chunk = data[i:i+RECORD_SIZE]
        if len(chunk) < RECORD_SIZE:
            break
        
        r = struct.unpack(RECORD_FORMAT, chunk)
        records.append({
            'timestamp': r[0],
            'worker_id': r[1],
            'fab_id': r[2],
            'edge_id': r[3],
            'veh_id': r[4],
            'enter_time': r[5],
            'exit_time': r[6],
            'edge_length': r[7],
            'edge_type': r[8],
        })
    
    # Polars DataFrame 생성
    df = pl.DataFrame(records)
    
    if len(df) > 0:
        # 계산 컬럼 추가
        df = df.with_columns([
            (pl.col('exit_time') - pl.col('enter_time')).alias('transit_time'),
            pl.col('edge_type').replace_strict(EDGE_TYPES, default='UNKNOWN').alias('edge_type_name'),
            (pl.col('timestamp') / 1000.0).alias('timestamp_sec'),
            (pl.col('enter_time') / 1000.0).alias('enter_time_sec'),
            (pl.col('exit_time') / 1000.0).alias('exit_time_sec'),
            pl.format('E{}', (pl.col('edge_id') + 1).cast(pl.Utf8).str.zfill(4)).alias('edge_id_fmt'),
        ])
        
        # Speed 계산 (0으로 나누기 방지)
        df = df.with_columns([
            pl.when(pl.col('transit_time') > 0)
              .then(pl.col('edge_length') / (pl.col('transit_time') / 1000.0))
              .otherwise(None)
              .alias('speed')
        ])
        
        # Edge ID 포맷 추가
        df = df.with_columns([
            (pl.col('edge_id') + 1).cast(pl.Utf8).str.zfill(4).str.concat('E').alias('edge_id_fmt')
        ])
        # 재정렬 (E0001, E0002, ...)
        df = df.with_columns([
            pl.format('E{}', (pl.col('edge_id') + 1).cast(pl.Utf8).str.zfill(4)).alias('edge_id_fmt')
        ])
    
    return df

print('Parser ready!')
print('Edge type별 기대 속도:', EXPECTED_SPEEDS)

In [ ]:
# 로그 파일 로드 (파일명 수정 필요)
# LOG_FILE = LOG_DIR / 'edge_transit_sim_xxx.bin'
LOG_FILE = log_files[0] if log_files else None  # 첫 번째 파일 사용

if LOG_FILE:
    df = parse_log_file(LOG_FILE)
    print(f'Loaded: {LOG_FILE.name}')
    print(f'Total records: {len(df):,}')
    print(f'\nDataFrame shape: {df.shape}')
    print(f'\nColumns: {df.columns}')
    print(f'\nSchema:\n{df.schema}')
    df.head(10)

## 2. Basic Statistics

In [ ]:
def print_summary(df, name='Dataset'):
    """데이터셋 요약 출력"""
    print(f'\n{"="*60}')
    print(f'{name} Summary')
    print(f'{"="*60}')
    
    print(f'\nTotal Records: {len(df):,}')
    print(f'Unique Vehicles: {df["veh_id"].n_unique()}')
    print(f'Unique Edges: {df["edge_id"].n_unique()}')
    print(f'Unique Fabs: {df["fab_id"].n_unique()}')
    print(f'Unique Workers: {df["worker_id"].n_unique()}')
    
    duration = (df['timestamp'].max() - df['timestamp'].min()) / 1000
    print(f'\nSimulation Duration: {duration:.1f} seconds')
    print(f'Throughput: {len(df) / duration:.1f} transits/sec')
    
    transit_stats = df.select([
        pl.col('transit_time').mean().alias('mean'),
        pl.col('transit_time').median().alias('median'),
        pl.col('transit_time').std().alias('std'),
        pl.col('transit_time').min().alias('min'),
        pl.col('transit_time').max().alias('max'),
    ]).row(0)
    
    print(f'\nTransit Time (ms):')
    print(f'  Mean: {transit_stats[0]:.1f}')
    print(f'  Median: {transit_stats[1]:.1f}')
    print(f'  Std: {transit_stats[2]:.1f}')
    print(f'  Min: {transit_stats[3]}')
    print(f'  Max: {transit_stats[4]}')
    
    speed_stats = df.select([
        pl.col('speed').drop_nulls().mean().alias('mean'),
        pl.col('speed').drop_nulls().median().alias('median'),
    ]).row(0)
    
    print(f'\nSpeed (m/s):')
    print(f'  Mean: {speed_stats[0]:.2f}')
    print(f'  Median: {speed_stats[1]:.2f}')

print_summary(df)

In [ ]:

# Fab별 통계
fab_stats = df.group_by('fab_id').agg([
    pl.col('veh_id').n_unique().alias('unique_vehicles'),
    pl.col('edge_id').n_unique().alias('unique_edges'),
    pl.len().alias('transit_count'),
    pl.col('timestamp').min().alias('timestamp_min'),
    pl.col('timestamp').max().alias('timestamp_max'),
    pl.col('transit_time').mean().alias('transit_mean'),
    pl.col('transit_time').median().alias('transit_median'),
    pl.col('transit_time').std().alias('transit_std'),
    pl.col('speed').mean().alias('speed_mean'),
]).with_columns([
    ((pl.col('timestamp_max') - pl.col('timestamp_min')) / 1000).alias('duration_sec'),
]).with_columns([
    (pl.col('transit_count') / pl.col('duration_sec')).alias('throughput'),
]).sort('fab_id')

print('\nFab-level Statistics:')
fab_stats


## 3. Throughput Analysis

In [ ]:

def plot_throughput_over_time(df, window_sec=1.0, by_fab=True):
    """시간에 따른 Throughput 시각화 (Plotly)"""
    
    # 전체 Throughput 계산
    df_sorted = df.sort('timestamp')
    min_time = df_sorted['timestamp_sec'].min()
    
    df_sorted = df_sorted.with_columns([
        ((pl.col('timestamp_sec') - min_time) / window_sec).floor().cast(pl.Int64).alias('time_bin')
    ])
    
    throughput = df_sorted.group_by('time_bin').agg([
        pl.len().alias('count')
    ]).sort('time_bin')
    
    throughput = throughput.with_columns([
        (pl.col('time_bin') * window_sec).alias('time_sec'),
        (pl.col('count') / window_sec).alias('throughput')
    ])
    
    # Subplots 생성
    if by_fab and df['fab_id'].n_unique() > 1:
        fig = make_subplots(
            rows=2, cols=1,
            subplot_titles=('Overall Throughput', 'Throughput by Fab'),
            vertical_spacing=0.12
        )
    else:
        fig = make_subplots(rows=1, cols=1)
    
    # 전체 Throughput
    mean_throughput = throughput['throughput'].mean()
    
    fig.add_trace(
        go.Scatter(
            x=throughput['time_sec'].to_list(),
            y=throughput['throughput'].to_list(),
            mode='lines',
            name='Throughput',
            fill='tozeroy',
            line=dict(color='blue', width=1),
        ),
        row=1, col=1
    )
    
    fig.add_hline(
        y=mean_throughput,
        line_dash='dash',
        line_color='red',
        annotation_text=f'Mean: {mean_throughput:.1f}',
        row=1, col=1
    )
    
    # Fab별 Throughput
    if by_fab and df['fab_id'].n_unique() > 1:
        for fab_id in sorted(df['fab_id'].unique().to_list()):
            fab_df = df_sorted.filter(pl.col('fab_id') == fab_id)
            fab_throughput = fab_df.group_by('time_bin').agg([
                pl.len().alias('count')
            ]).sort('time_bin').with_columns([
                (pl.col('time_bin') * window_sec).alias('time_sec'),
                (pl.col('count') / window_sec).alias('throughput')
            ])
            
            fig.add_trace(
                go.Scatter(
                    x=fab_throughput['time_sec'].to_list(),
                    y=fab_throughput['throughput'].to_list(),
                    mode='lines',
                    name=f'Fab {fab_id}',
                    line=dict(width=1),
                ),
                row=2, col=1
            )
    
    # Layout 설정
    fig.update_xaxes(title_text='Time (seconds)', row=1, col=1)
    fig.update_yaxes(title_text='Throughput (transits/sec)', row=1, col=1)
    
    if by_fab and df['fab_id'].n_unique() > 1:
        fig.update_xaxes(title_text='Time (seconds)', row=2, col=1)
        fig.update_yaxes(title_text='Throughput (transits/sec)', row=2, col=1)
    
    fig.update_layout(
        height=800 if by_fab and df['fab_id'].n_unique() > 1 else 400,
        showlegend=True,
        title_text=f'Throughput Analysis (window={window_sec}s)'
    )
    
    fig.show()

plot_throughput_over_time(df, window_sec=1.0)


## 4. Transit Time Analysis

In [ ]:
def plot_transit_time_distribution(df):
    """Transit Time 분포 시각화 (Plotly)"""
    
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Transit Time Distribution',
            'Transit Time by Edge Type',
            'Transit Time by Fab',
            'Speed Distribution'
        ),
        specs=[[{'type': 'histogram'}, {'type': 'box'}],
               [{'type': 'box'}, {'type': 'histogram'}]]
    )
    
    # 1. 전체 분포 (히스토그램)
    transit_times = df['transit_time'].to_list()
    mean_transit = df['transit_time'].mean()
    median_transit = df['transit_time'].median()
    
    fig.add_trace(
        go.Histogram(
            x=transit_times,
            nbinsx=50,
            name='Transit Time',
            marker_color='lightblue',
            marker_line_color='black',
            marker_line_width=0.5,
        ),
        row=1, col=1
    )
    
    fig.add_vline(
        x=mean_transit,
        line_dash='dash',
        line_color='red',
        annotation_text=f'Mean: {mean_transit:.1f}ms',
        row=1, col=1
    )
    
    fig.add_vline(
        x=median_transit,
        line_dash='dash',
        line_color='green',
        annotation_text=f'Median: {median_transit:.1f}ms',
        row=1, col=1
    )
    
    # 2. Edge Type별 분포
    edge_types = df.select(pl.col('edge_type_name').drop_nulls().unique()).to_series().to_list()
    for edge_type in edge_types:
        transit_data = df.filter(pl.col('edge_type_name') == edge_type)['transit_time'].to_list()
        fig.add_trace(
            go.Box(
                y=transit_data,
                name=edge_type,
                boxmean='sd'
            ),
            row=1, col=2
        )
    
    # 3. Fab별 분포
    if df['fab_id'].n_unique() > 1:
        for fab_id in sorted(df['fab_id'].unique().to_list()):
            transit_data = df.filter(pl.col('fab_id') == fab_id)['transit_time'].to_list()
            fig.add_trace(
                go.Box(
                    y=transit_data,
                    name=f'Fab {fab_id}',
                    boxmean='sd'
                ),
                row=2, col=1
            )
    else:
        fig.add_annotation(
            text='Single Fab',
            xref='x3', yref='y3',
            x=0.5, y=0.5,
            showarrow=False,
            row=2, col=1
        )
    
    # 4. Speed 분포
    valid_speeds = df.select(pl.col('speed').drop_nulls())['speed'].to_list()
    mean_speed = df.select(pl.col('speed').drop_nulls().mean()).item()
    
    fig.add_trace(
        go.Histogram(
            x=valid_speeds,
            nbinsx=50,
            name='Speed',
            marker_color='lightcoral',
            marker_line_color='black',
            marker_line_width=0.5,
        ),
        row=2, col=2
    )
    
    fig.add_vline(
        x=mean_speed,
        line_dash='dash',
        line_color='red',
        annotation_text=f'Mean: {mean_speed:.2f}m/s',
        row=2, col=2
    )
    
    # Layout 설정
    fig.update_xaxes(title_text='Transit Time (ms)', row=1, col=1)
    fig.update_yaxes(title_text='Count', row=1, col=1)
    
    fig.update_xaxes(title_text='Edge Type', row=1, col=2)
    fig.update_yaxes(title_text='Transit Time (ms)', row=1, col=2)
    
    fig.update_xaxes(title_text='Fab', row=2, col=1)
    fig.update_yaxes(title_text='Transit Time (ms)', row=2, col=1)
    
    fig.update_xaxes(title_text='Speed (m/s)', row=2, col=2)
    fig.update_yaxes(title_text='Count', row=2, col=2)
    
    fig.update_layout(
        height=900,
        showlegend=True,
        title_text='Transit Time Analysis'
    )
    
    fig.show()

plot_transit_time_distribution(df)

## 5. Edge Congestion Analysis

In [ ]:

def analyze_edge_congestion(df, top_n=20):
    """Edge별 혼잡도 분석 - Edge Type별 기대 속도 반영"""
    
    edge_stats = df.group_by('edge_id').agg([
        pl.len().alias('transit_count'),
        pl.col('transit_time').mean().alias('transit_mean'),
        pl.col('transit_time').median().alias('transit_median'),
        pl.col('transit_time').std().alias('transit_std'),
        pl.col('transit_time').max().alias('transit_max'),
        pl.col('speed').mean().alias('speed_mean'),
        pl.col('edge_length').first().alias('edge_length'),
        pl.col('edge_type_name').first().alias('edge_type'),
    ])
    
    # Edge ID 포맷 생성 (숫자 edge_id 기반)
    edge_stats = edge_stats.with_columns([
        pl.format('E{}', (pl.col('edge_id') + 1).cast(pl.Utf8).str.zfill(4)).alias('edge_id_fmt')
    ])
    
    # Edge Type별 기대 속도 매핑 (when-then 체인 사용)
    expected_speed_expr = pl.col('edge_type')
    for type_name, speed in EXPECTED_SPEEDS.items():
        if type_name == 'LINEAR':
            expected_speed_expr = pl.when(pl.col('edge_type') == type_name).then(pl.lit(speed))
        else:
            expected_speed_expr = expected_speed_expr.when(pl.col('edge_type') == type_name).then(pl.lit(speed))
    expected_speed_expr = expected_speed_expr.otherwise(pl.lit(3.0)).alias('expected_speed')
    
    edge_stats = edge_stats.with_columns([
        expected_speed_expr
    ]).with_columns([
        (pl.col('edge_length') / pl.col('expected_speed') * 1000).alias('expected_time')
    ]).with_columns([
        (pl.col('transit_count').cast(pl.Float64) * (pl.col('transit_mean') / pl.col('expected_time'))).alias('congestion_score')
    ])
    
    edge_stats_sorted = edge_stats.sort('congestion_score', descending=True)
    
    print(f'\nTop {top_n} Congested Edges:')
    print(edge_stats_sorted.head(top_n))
    
    # 시각화
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('Edge Usage Distribution', f'Top {top_n} Congested Edges'),
        column_widths=[0.4, 0.6]
    )
    
    # 1. 통과 횟수 분포
    transit_counts = edge_stats['transit_count'].to_list()
    fig.add_trace(
        go.Histogram(
            x=transit_counts,
            nbinsx=30,
            marker_color='lightblue',
            marker_line_color='black',
            marker_line_width=0.5,
            name='Transit Count'
        ),
        row=1, col=1
    )
    
    # 2. Top N 혼잡 Edge
    top_edges = edge_stats_sorted.head(top_n)
    edge_labels = top_edges['edge_id_fmt'].to_list()
    congestion_scores = top_edges['congestion_score'].to_list()
    
    fig.add_trace(
        go.Bar(
            y=edge_labels[::-1],  # 역순으로 표시 (높은 것이 위로)
            x=congestion_scores[::-1],
            orientation='h',
            marker_color='coral',
            text=[f'{s:.1f}' for s in congestion_scores[::-1]],
            textposition='outside',
            name='Congestion Score'
        ),
        row=1, col=2
    )
    
    # Layout 설정
    fig.update_xaxes(title_text='Transit Count', row=1, col=1)
    fig.update_yaxes(title_text='Number of Edges', row=1, col=1)
    
    fig.update_xaxes(title_text='Congestion Score', row=1, col=2)
    fig.update_yaxes(title_text='Edge ID', row=1, col=2)
    
    fig.update_layout(
        height=600,
        showlegend=False,
        title_text='Edge Congestion Analysis (Edge Type별 기대 속도 반영)'
    )
    
    fig.show()
    
    return edge_stats_sorted

edge_congestion = analyze_edge_congestion(df)


## 6. Vehicle Utilization

In [ ]:

def analyze_vehicle_utilization(df):
    """Vehicle별 활용률 분석"""
    
    veh_stats = df.group_by('veh_id').agg([
        pl.len().alias('transit_count'),
        pl.col('timestamp').min().alias('first_time'),
        pl.col('timestamp').max().alias('last_time'),
        pl.col('transit_time').sum().alias('total_transit_time'),
        pl.col('transit_time').mean().alias('avg_transit_time'),
        pl.col('edge_length').sum().alias('total_distance'),
        pl.col('fab_id').first().alias('fab_id'),
    ])
    
    veh_stats = veh_stats.with_columns([
        (pl.col('last_time') - pl.col('first_time')).alias('active_duration')
    ]).with_columns([
        (pl.col('total_transit_time') / pl.col('active_duration') * 100).clip(0, 100).alias('utilization')
    ])
    
    print('\nVehicle Utilization Summary:')
    print(f'  Mean Utilization: {veh_stats["utilization"].mean():.1f}%')
    print(f'  Median Utilization: {veh_stats["utilization"].median():.1f}%')
    print(f'  Total Distance (all vehicles): {veh_stats["total_distance"].sum():.1f}m')
    
    # 시각화
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Vehicle Utilization Distribution',
            'Transits per Vehicle',
            'Vehicle Utilization by Fab',
            'Total Distance per Vehicle'
        )
    )
    
    # 1. 활용률 분포
    utilizations = veh_stats['utilization'].to_list()
    mean_util = veh_stats['utilization'].mean()
    
    fig.add_trace(
        go.Histogram(
            x=utilizations,
            nbinsx=30,
            marker_color='lightblue',
            marker_line_color='black',
            marker_line_width=0.5,
        ),
        row=1, col=1
    )
    
    fig.add_vline(
        x=mean_util,
        line_dash='dash',
        line_color='red',
        row=1, col=1
    )
    
    # 2. 통과 횟수 분포
    transit_counts = veh_stats['transit_count'].to_list()
    
    fig.add_trace(
        go.Histogram(
            x=transit_counts,
            nbinsx=30,
            marker_color='lightgreen',
            marker_line_color='black',
            marker_line_width=0.5,
        ),
        row=1, col=2
    )
    
    # 3. Fab별 활용률
    if df['fab_id'].n_unique() > 1:
        fab_util = veh_stats.group_by('fab_id').agg([
            pl.col('utilization').mean().alias('mean_utilization')
        ]).sort('fab_id')
        
        fig.add_trace(
            go.Bar(
                x=fab_util['fab_id'].to_list(),
                y=fab_util['mean_utilization'].to_list(),
                marker_color='lightcoral',
                text=[f'{v:.1f}%' for v in fab_util['mean_utilization'].to_list()],
                textposition='outside',
            ),
            row=2, col=1
        )
    else:
        fig.add_annotation(
            text='Single Fab',
            xref='x3', yref='y3',
            x=0.5, y=0.5,
            showarrow=False,
            row=2, col=1
        )
    
    # 4. 총 이동거리 분포
    total_distances = veh_stats['total_distance'].to_list()
    
    fig.add_trace(
        go.Histogram(
            x=total_distances,
            nbinsx=30,
            marker_color='lightyellow',
            marker_line_color='black',
            marker_line_width=0.5,
        ),
        row=2, col=2
    )
    
    # Layout 설정
    fig.update_xaxes(title_text='Utilization (%)', row=1, col=1)
    fig.update_yaxes(title_text='Number of Vehicles', row=1, col=1)
    
    fig.update_xaxes(title_text='Transit Count', row=1, col=2)
    fig.update_yaxes(title_text='Number of Vehicles', row=1, col=2)
    
    fig.update_xaxes(title_text='Fab ID', row=2, col=1)
    fig.update_yaxes(title_text='Mean Utilization (%)', row=2, col=1)
    
    fig.update_xaxes(title_text='Total Distance (m)', row=2, col=2)
    fig.update_yaxes(title_text='Number of Vehicles', row=2, col=2)
    
    fig.update_layout(
        height=900,
        showlegend=False,
        title_text='Vehicle Utilization Analysis'
    )
    
    fig.show()
    
    return veh_stats

veh_utilization = analyze_vehicle_utilization(df)


## 7. Compare Multiple Experiments

In [ ]:
def load_multiple_experiments(file_paths, names=None):
    """
    여러 실험 결과 로드
    
    Args:
        file_paths: 로그 파일 경로 리스트
        names: 실험 이름 리스트 (없으면 파일명 사용)
    """
    experiments = {}
    
    for i, path in enumerate(file_paths):
        path = Path(path)
        name = names[i] if names else path.stem
        
        df = parse_log_file(path)
        experiments[name] = df
        print(f'Loaded {name}: {len(df):,} records')
    
    return experiments

def compare_experiments(experiments):
    """실험 결과 비교 (Plotly)"""
    comparison_data = []
    
    for name, df in experiments.items():
        duration = (df['timestamp'].max() - df['timestamp'].min()) / 1000
        comparison_data.append({
            'experiment': name,
            'records': len(df),
            'duration_sec': duration,
            'throughput': len(df) / duration,
            'transit_mean': df['transit_time'].mean(),
            'transit_median': df['transit_time'].median(),
            'speed_mean': df['speed'].mean(),
            'unique_vehicles': df['veh_id'].n_unique(),
            'unique_edges': df['edge_id'].n_unique(),
        })
    
    comp_df = pl.DataFrame(comparison_data)
    print('\nExperiment Comparison:')
    print(comp_df)
    
    # 시각화
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Throughput Comparison',
            'Transit Time Comparison',
            'Average Speed Comparison',
            'Transit Time Distribution'
        )
    )
    
    exp_names = list(experiments.keys())
    
    # 1. Throughput 비교
    fig.add_trace(
        go.Bar(
            x=comp_df['experiment'].to_list(),
            y=comp_df['throughput'].to_list(),
            marker_color='lightblue',
            text=[f'{v:.1f}' for v in comp_df['throughput'].to_list()],
            textposition='outside',
        ),
        row=1, col=1
    )
    
    # 2. Transit Time 비교 (Mean vs Median)
    fig.add_trace(
        go.Bar(
            x=comp_df['experiment'].to_list(),
            y=comp_df['transit_mean'].to_list(),
            name='Mean',
            marker_color='lightcoral',
        ),
        row=1, col=2
    )
    
    fig.add_trace(
        go.Bar(
            x=comp_df['experiment'].to_list(),
            y=comp_df['transit_median'].to_list(),
            name='Median',
            marker_color='lightgreen',
        ),
        row=1, col=2
    )
    
    # 3. Speed 비교
    fig.add_trace(
        go.Bar(
            x=comp_df['experiment'].to_list(),
            y=comp_df['speed_mean'].to_list(),
            marker_color='lightyellow',
            text=[f'{v:.2f}' for v in comp_df['speed_mean'].to_list()],
            textposition='outside',
        ),
        row=2, col=1
    )
    
    # 4. Transit Time 분포 비교 (Box plot)
    for name in exp_names:
        transit_data = experiments[name]['transit_time'].to_list()
        fig.add_trace(
            go.Box(
                y=transit_data,
                name=name,
                boxmean='sd'
            ),
            row=2, col=2
        )
    
    # Layout 설정
    fig.update_xaxes(title_text='Experiment', row=1, col=1)
    fig.update_yaxes(title_text='Throughput (transits/sec)', row=1, col=1)
    
    fig.update_xaxes(title_text='Experiment', row=1, col=2)
    fig.update_yaxes(title_text='Transit Time (ms)', row=1, col=2)
    
    fig.update_xaxes(title_text='Experiment', row=2, col=1)
    fig.update_yaxes(title_text='Speed (m/s)', row=2, col=1)
    
    fig.update_xaxes(title_text='Experiment', row=2, col=2)
    fig.update_yaxes(title_text='Transit Time (ms)', row=2, col=2)
    
    fig.update_layout(
        height=900,
        showlegend=True,
        title_text='Experiment Comparison'
    )
    
    fig.show()
    
    return comp_df

# 예시: 여러 실험 비교
# experiments = load_multiple_experiments([
#     LOG_DIR / 'experiment_baseline.bin',
#     LOG_DIR / 'experiment_new_routing.bin',
#     LOG_DIR / 'experiment_optimized.bin',
# ], names=['Baseline', 'New Routing', 'Optimized'])
# compare_experiments(experiments)

## 8. Export Results

In [ ]:
# CSV로 내보내기
OUTPUT_DIR = Path('/content/drive/MyDrive/vps_logs/output')
OUTPUT_DIR.mkdir(exist_ok=True)

# 전체 데이터
# df.write_csv(OUTPUT_DIR / 'full_data.csv')

# 요약 통계
# fab_stats.write_csv(OUTPUT_DIR / 'fab_stats.csv')
# edge_congestion.write_csv(OUTPUT_DIR / 'edge_congestion.csv')
# veh_utilization.write_csv(OUTPUT_DIR / 'vehicle_utilization.csv')

print('Export examples (uncomment to use):')
print('  df.write_csv(OUTPUT_DIR / "full_data.csv")')
print('  fab_stats.write_csv(OUTPUT_DIR / "fab_stats.csv")')
print('  edge_congestion.write_csv(OUTPUT_DIR / "edge_congestion.csv")')
print('  veh_utilization.write_csv(OUTPUT_DIR / "vehicle_utilization.csv")')

## 9. Custom Analysis

자유롭게 분석 코드를 추가하세요.

In [ ]:
# 여기에 커스텀 분석 코드 작성
# 예시:
#
# # 특정 Fab만 필터링
# fab0_df = df.filter(pl.col('fab_id') == 0)
#
# # 특정 시간대만 필터링
# start_time = 10000  # 10초 이후
# end_time = 60000    # 60초까지
# time_filtered = df.filter(
#     (pl.col('timestamp') >= start_time) & (pl.col('timestamp') <= end_time)
# )
#
# # Edge Type별 평균 속도
# df.group_by('edge_type_name').agg([
#     pl.col('speed').mean().alias('avg_speed')
# ])
#
# # 혼잡한 Edge들만 필터링해서 자세히 보기
# congested_edges = edge_congestion.head(10)['edge_id'].to_list()
# congested_df = df.filter(pl.col('edge_id').is_in(congested_edges))
#
# # Lazy API로 대용량 데이터 처리 (메모리 효율적)
# lazy_df = pl.scan_csv('large_file.csv')  # 파일을 바로 읽지 않고 lazy하게
# result = lazy_df.filter(pl.col('fab_id') == 0).group_by('edge_id').agg([
#     pl.col('transit_time').mean()
# ]).collect()  # collect()를 호출할 때 실제 계산 수행

## 10. Polars 대용량 데이터 처리 팁

### 구글 코랩에서 대용량 로그 분석하기

Polars는 pandas보다 **훨씬 빠르고 메모리 효율적**입니다:

- **속도**: Rust 기반으로 멀티코어를 완전히 활용
- **메모리**: pandas보다 2-5배 적은 메모리 사용
- **Lazy evaluation**: 필요한 연산만 수행
- **Streaming**: 메모리보다 큰 데이터도 처리 가능

### 구글 코랩 무료 vs Pro

- **무료**: RAM ~12GB, 데이터 크기 제한적
- **Pro ($10/월)**: RAM ~25GB, GPU 사용 가능
- **Pro+**: RAM ~50GB, 대용량 데이터 처리 가능

### 권장사항

- **소규모 (~100MB)**: 코랩 무료 충분
- **중규모 (100MB-1GB)**: Polars + 코랩 무료로 가능
- **대규모 (1GB+)**: Polars streaming + 코랩 Pro 권장
- **초대규모 (10GB+)**: 로컬 환경 권장 (RAM 32GB+)

In [ ]:
# 메모리 사용량 체크
import psutil

def check_memory_usage():
    """현재 메모리 사용량 확인"""
    process = psutil.Process()
    mem_info = process.memory_info()
    mem_mb = mem_info.rss / 1024 / 1024
    
    # 시스템 전체 메모리
    vm = psutil.virtual_memory()
    total_mb = vm.total / 1024 / 1024
    available_mb = vm.available / 1024 / 1024
    used_percent = vm.percent
    
    print(f'현재 프로세스 메모리 사용량: {mem_mb:.1f} MB')
    print(f'시스템 총 메모리: {total_mb:.1f} MB')
    print(f'사용 가능 메모리: {available_mb:.1f} MB')
    print(f'메모리 사용률: {used_percent:.1f}%')
    
    if used_percent > 80:
        print('⚠️  메모리 사용률이 높습니다! 다음을 고려하세요:')
        print('  1. 불필요한 변수 삭제: del large_df')
        print('  2. 필요한 컬럼만 선택: df.select([...필요한 컬럼...])')
        print('  3. 데이터 필터링: df.filter(...)')
        print('  4. 코랩 Pro로 업그레이드')
    
    return {
        'process_mb': mem_mb,
        'total_mb': total_mb,
        'available_mb': available_mb,
        'used_percent': used_percent
    }

# DataFrame 크기 확인
def check_dataframe_size(df, name='df'):
    """DataFrame의 메모리 사용량 확인"""
    # Polars DataFrame의 메모리 사용량 추정
    mem_bytes = df.estimated_size()
    mem_mb = mem_bytes / 1024 / 1024
    
    print(f'{name} 크기: {len(df):,} rows × {len(df.columns)} cols')
    print(f'{name} 메모리: {mem_mb:.1f} MB')
    
    return mem_mb

print('=== 메모리 사용량 체크 ===\n')
mem_info = check_memory_usage()
print('\n=== DataFrame 크기 ===\n')
df_size = check_dataframe_size(df, 'df')

# 최적화 팁
print('\n=== Polars 최적화 팁 ===\n')
print('1. 필요한 컬럼만 로드:')
print('   df.select(["edge_id", "transit_time", "speed"])')
print('\n2. Lazy API 사용 (대용량 데이터):')
print('   lazy_df = pl.scan_parquet("large_file.parquet")')
print('   result = lazy_df.filter(...).group_by(...).agg(...).collect()')
print('\n3. 메모리 절약을 위한 타입 변환:')
print('   df.with_columns([pl.col("edge_id").cast(pl.UInt16)])  # Int32 → UInt16')
print('\n4. 청크 단위로 처리:')
print('   for chunk in pl.read_csv_batched("large.csv", batch_size=10000):')
print('       process(chunk)')